In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'country_demo', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())

print(emission_df.head())
results_temp = session.query(Temp_change)

#print(results) 

temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']

                       Entity  Code  Year  AnnualCO2emissions
0                      Africa  None  1961           161437399
1                        Asia  None  1961          1392745886
2  Asia (excl. China & India)  None  1961           721204193
3                       EU-27  None  1961          2157634122
4                       EU-28  None  1961          2746248923


In [7]:
season_df = temp_df.loc[temp_df["Months"].isin(selection)]

#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]


# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
meteor_df_new = meteor_df.copy()

# Calculate avg temp per Meteorological year
meteor_df_new['avg_temp']= round(meteor_df_new.mean(axis =1),3)
meteor_id_df = meteor_df_new.set_index('Area')

session.close() 
emission_df


,Entity,Code,Year,AnnualCO2emissions
0,Africa,None,1961,161437399
1,Asia,None,1961,1392745886
2,Asia (excl. China & India),None,1961,721204193
3,EU-27,None,1961,2157634122
4,EU-28,None,1961,2746248923
...,...,...,...,...
13801,Samoa,WSM,2019,285379
13802,Yemen,YEM,2019,10255048
13803,South Africa,ZAF,2019,478608101
13804,Zambia,ZMB,2019,6720490


In [8]:
#calculate overall avg co2 emission per country
avg_co2 =  emission_df.groupby("Entity").agg({'AnnualCO2emissions':'mean'})
avg_co2 = round(avg_co2/1000000,3) ## converting GT to Mega ton for the tooltip


In [9]:
#Merge Temp_change by meteor year per country to Avg_Co2 Emission df
merged_co2_country =meteor_id_df.merge(avg_co2, how = 'inner',  left_index=True, right_index=True,)
merged_co2_country

,field1,Months,Element,Unit,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp,AnnualCO2emissions
Afghanistan,32,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,0.176,...,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958,3.036
Albania,66,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,0.551,...,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577,4.503
Algeria,100,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,0.404,...,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366,74.340
Andorra,168,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,0.403,...,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480,0.246
Angola,202,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,0.193,...,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773,11.971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
South America,8804,Meteorological year,Temperature change,°C,0.116,-0.139,0.139,-0.217,-0.015,0.007,...,0.943,0.834,1.105,1.435,1.242,1.232,1.034,1.310,147.178,649.369
Asia,8838,Meteorological year,Temperature change,°C,0.049,-0.008,0.266,-0.166,-0.033,0.326,...,0.693,1.054,0.909,1.284,1.457,1.464,1.343,1.369,147.765,8250.110
Europe,9042,Meteorological year,Temperature change,°C,0.874,0.300,0.103,-0.523,-0.430,0.724,...,1.789,1.118,1.807,2.106,2.286,1.959,2.009,2.116,151.420,6400.192
Oceania,9212,Meteorological year,Temperature change,°C,0.116,0.140,-0.118,-0.030,0.067,-0.231,...,0.276,1.396,1.083,0.997,1.163,1.110,1.096,1.370,153.948,306.085


In [10]:

#find null
merged_co2_country.isna().sum()




field1                0
Months                0
Element               0
Unit                  0
1961                  0
                     ..
2017                  0
2018                  0
2019                  0
avg_temp              0
AnnualCO2emissions    0
Length: 65, dtype: int64

In [11]:
#Get New Countries from the merged DF
New_Countires = merged_co2_country.index

#Create a dictionary holding above values
 #meta ={
#    'country' : country_name,
#    'demo_info' : [web scraped data],
#    'tool_tip' : [{'c_name': country_name,
#                'avg_temp':avg_temp,
#                'avg_co2' : new_Avg_c02,
#                'population':population from web scraping
#                }]
#   } 

#New Code---------------------------------------------------

meta = []

for country in New_Countires:
    temp_co2_obj = {
                    "Country":country,
                    "Avg Temp Change":merged_co2_country.loc[country,"avg_temp"],
                    "Avg Co2 Change":merged_co2_country.loc[country,"AnnualCO2emissions"],
                    }
    meta.append(temp_co2_obj)




meta

[{'Country': 'Afghanistan', 'Avg Temp Change': 0.958, 'Avg Co2 Change': 3.036},
 {'Country': 'Albania', 'Avg Temp Change': 1.577, 'Avg Co2 Change': 4.503},
 {'Country': 'Algeria', 'Avg Temp Change': 2.366, 'Avg Co2 Change': 74.34},
 {'Country': 'Andorra', 'Avg Temp Change': 3.48, 'Avg Co2 Change': 0.246},
 {'Country': 'Angola', 'Avg Temp Change': 3.773, 'Avg Co2 Change': 11.971},
 {'Country': 'Anguilla', 'Avg Temp Change': 4.185, 'Avg Co2 Change': 0.056},
 {'Country': 'Argentina', 'Avg Temp Change': 5.894, 'Avg Co2 Change': 124.014},
 {'Country': 'Australia', 'Avg Temp Change': 7.76, 'Avg Co2 Change': 273.351},
 {'Country': 'Austria', 'Avg Temp Change': 8.636, 'Avg Co2 Change': 59.374},
 {'Country': 'Bahamas', 'Avg Temp Change': 9.45, 'Avg Co2 Change': 2.751},
 {'Country': 'Bahrain', 'Avg Temp Change': 10.288, 'Avg Co2 Change': 14.123},
 {'Country': 'Bangladesh',
  'Avg Temp Change': 10.391,
  'Avg Co2 Change': 24.771},
 {'Country': 'Belize', 'Avg Temp Change': 13.49, 'Avg Co2 Change':